In [22]:
import pandas as pd
from meteostat import Stations, Point, Monthly
from datetime import datetime
import time
import numpy as np

In [10]:
df = pd.read_csv("ficheros/03-areas_country.csv", index_col=0)
df.head()

,latitude,longitude,town,region,country,price,water,wc,grey_water
0,39.06956,-0.40980,Carcaixent,Comunitat Valenciana,España,Free,yes,yes,yes
1,42.64159,-8.89696,Boiro,Galicia,España,Free,yes,yes,unknown
2,42.90715,0.35921,Arreau,Occitanie,France,Paying,yes,no,yes
3,38.39479,-0.41007,el Campello,Comunitat Valenciana,España,Paying,yes,yes,unknown
4,43.37179,-8.44443,A Coruña,Galicia,España,Free,yes,yes,unknown


In [16]:
def weather_month(dataf):

    data_list = []
    for ind, row in dataf.iterrows():
        lat = row["latitude"]
        lon = row["longitude"]

        stations = Stations()
        stations = stations.nearby(lat,lon)
        station = stations.fetch(1)
        station_name = station.index[0]

        start = datetime(2022, 1, 1)
        end = datetime(2022, 12, 31)

        data = Monthly(station_name, start, end)
        data = data.fetch()
        data.reset_index(inplace=True)

        average_temp_month = data[["tavg"]].T
        df_temp = average_temp_month.reset_index(drop= True)
        df_temp["latitude"] = lat
        df_temp["longitude"] = lon
        data_list.append(df_temp)
        
    dat = pd.concat(data_list, axis=0, ignore_index= True)

    return dat

In [17]:
df_weather = weather_month(df)

In [19]:
lista_col = {0: "Jan", 1: "Feb", 2: "Mar", 3: "Apr", 4: "May", 5: "Jun",
            6: "Jul", 7: "Aug", 8: "Sep", 9: "Oct", 10: "Nov", 11: "Dec"}

In [20]:
df_weather.rename(columns=lista_col, inplace= True)
df_weather.head(2)

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,latitude,longitude
0,10.4,12.8,12.9,14.2,NaN,24.5,26.8,27.8,23.9,19.9,16.3,13.6,39.06956,-0.40980
1,10.2,11.2,12.1,12.7,17.8,18.0,23.0,21.0,18.9,17.3,12.8,12.0,42.64159,-8.89696


In [21]:
df_weather.isnull().sum()

Jan           50
Feb           50
Mar           77
Apr           53
May           58
Jun           56
Jul           61
Aug           65
Sep           84
Oct          103
Nov          259
Dec          498
latitude       0
longitude      0
dtype: int64

In [24]:
df_weather[df_weather["Dec"].isnull()]

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,latitude,longitude
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51.28586,4.50299
11,4.4,7.4,11.4,17.2,19.8,22.7,23.1,17.0,16.2,9.8,5.6,NaN,47.06500,0.31783
12,2.9,7.4,12.0,18.8,21.4,23.9,24.5,18.4,18.2,NaN,NaN,NaN,45.05802,1.65895
14,2.9,7.4,12.0,18.8,21.4,23.9,24.5,18.4,18.2,NaN,NaN,NaN,45.05397,1.58078
15,3.6,5.6,5.3,8.4,16.3,20.6,20.7,21.2,14.2,NaN,NaN,NaN,51.04677,13.74581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
901,3.6,6.7,10.0,17.4,22.3,24.9,22.2,17.6,14.9,8.2,NaN,NaN,42.46602,1.49422
903,7.3,9.4,11.8,18.6,20.8,21.2,18.3,17.7,14.1,8.1,NaN,NaN,46.35049,-1.43666
905,3.0,6.9,9.0,11.1,17.9,20.8,22.3,22.2,16.6,17.6,NaN,NaN,45.86873,3.48255
907,5.9,10.1,13.4,19.9,25.6,28.3,26.5,21.4,18.2,12.1,9.2,NaN,43.78299,6.44211


In [28]:
df_weather[df_weather.isnull().any(axis= 1)]

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,latitude,longitude
0,10.4,12.8,12.9,14.2,NaN,24.5,26.8,27.8,23.9,19.9,16.3,13.6,39.06956,-0.40980
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51.28586,4.50299
11,4.4,7.4,11.4,17.2,19.8,22.7,23.1,17.0,16.2,9.8,5.6,NaN,47.06500,0.31783
12,2.9,7.4,12.0,18.8,21.4,23.9,24.5,18.4,18.2,NaN,NaN,NaN,45.05802,1.65895
14,2.9,7.4,12.0,18.8,21.4,23.9,24.5,18.4,18.2,NaN,NaN,NaN,45.05397,1.58078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
901,3.6,6.7,10.0,17.4,22.3,24.9,22.2,17.6,14.9,8.2,NaN,NaN,42.46602,1.49422
903,7.3,9.4,11.8,18.6,20.8,21.2,18.3,17.7,14.1,8.1,NaN,NaN,46.35049,-1.43666
905,3.0,6.9,9.0,11.1,17.9,20.8,22.3,22.2,16.6,17.6,NaN,NaN,45.86873,3.48255
907,5.9,10.1,13.4,19.9,25.6,28.3,26.5,21.4,18.2,12.1,9.2,NaN,43.78299,6.44211


In [33]:
df_weather.to_csv("ficheros/04-weather_coordinates.csv")